In [49]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [92]:
#Test webu, vytvoreni nadpisu tabulky a zapsani prvnich metadat 
url = "https://wiki.uibk.ac.at/noscemus/A_Latin_Letter_containing_some_Animadversions_upon_Mr._Isaac_Newton,_his_Theory_of_Light"
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find("table", class_="wikitable")
    data = [item.get_text(strip=True) for item in table.find_all("td")]
    indices = [item.get_text(strip=True) for item in table.find_all("th")]
    metadata_tabulka = pd.DataFrame(index=indices)
else:
    print("Chyba, response code je:", response.status_code)

In [93]:
metadata_tabulka

""
Author
Full title
In
Year
Place
Publisher/Printer
Era
Form/Genre
Discipline/Content
Digital copies


In [84]:
# vytvoreni seznamu webovych stranek noscemu, na zaklade ktereho iteruji
url = ["https://wiki.uibk.ac.at/noscemus/_-_/index.php?title=Category:Works&pageuntil=De+curandis+vulneribus+sclopettorum#mw-pages", "https://wiki.uibk.ac.at/noscemus/_-_/index.php?title=Category:Works&pagefrom=De+curandis+vulneribus+sclopettorum#mw-pages", "https://wiki.uibk.ac.at/noscemus/_-_/index.php?title=Category:Works&pagefrom=Discursus+astronomicus+novissimus#mw-pages", "https://wiki.uibk.ac.at/noscemus/_-_/index.php?title=Category:Works&pagefrom=In+opus+revolutionum+Nicolai+Copernici+Torunnaei+dialogus#mw-pages", "https://wiki.uibk.ac.at/noscemus/_-_/index.php?title=Category:Works&pagefrom=Petri+Nonii+Salaciensis+opera#mw-pages",]
seznam =[]
for item in url:
    response = requests.get(item)
    soup = BeautifulSoup(response.content, "html.parser")
    tag = soup.find("div", class_="mw-category")
    for item in re.findall("(?<=href=\").*(?=\" title)", str(tag)):
        seznam.append(item) 

['/noscemus/A_Latin_Letter_containing_some_Animadversions_upon_Mr._Isaac_Newton,_his_Theory_of_Light', '/noscemus/Acarnania', '/noscemus/Ad_astronautas_Americanos', '/noscemus/Ad_Burnettum_sacrae_theoriae_telluris_auctorem', '/noscemus/Ad_Clusii_nomen_lusus', '/noscemus/Ad_Gilbertum', '/noscemus/Ad_Gulielmum_Rondeletium', '/noscemus/Ad_Iordanum_Brunum', '/noscemus/Ad_Laurentium_Massam_pro_Avicenna_ode', '/noscemus/Ad_lectorem_trigonometriae_studiosum', '/noscemus/Ad_Simonem_Paulli', '/noscemus/Additamentum_de_natura_scientiarum_mathematicarum', '/noscemus/Adulatio_perniciosa', '/noscemus/Aerarium_philosophiae_mathematicae', '/noscemus/Aetnae_topographia', '/noscemus/Alchemia_Libavii', '/noscemus/Alchemiae_quam_vocant_artisque_metallicae_doctrina', '/noscemus/Alea', '/noscemus/Algorithmus_minutiarum_vulgarium', '/noscemus/Almagestum_novum', '/noscemus/Almanach_novum_atque_correctum_super_anno_Domini_1510', '/noscemus/Amoenitates_academicae', '/noscemus/Amoenitates_exoticae', '/noscemus/

In [ ]:
len(seznam)

In [95]:
#zapsani metadat ze vsech zaznamu v noscemus do pd.dataframu "metadata_tabulka"
for page in seznam:
    url = "https://wiki.uibk.ac.at"+page
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find("table", class_="wikitable")
    data = [item.get_text(strip=True) for item in table.find_all("td")]
    metadata_tabulka = pd.DataFrame([data])
metadata_tabulka

KeyboardInterrupt: 